In [13]:
import pandas as pd
import json
import re

In [15]:
with open('./files/服饰数据.json', 'r')as f:
    data = f.readlines()

In [19]:
for idx, d in enumerate(data):
    if '沐浴露' in d:
        print(idx,d)
        break

249769         "ocr": "色泽明朗，细节，产品信，色泽清晰明朗，在阳光的照耀下更显夺目，的热情活泼，大胆创新的融合了缤纷，给人以良好的印，材料与设计相配合，时尚与品牌相融合，把你的品位细节彰显得淋漓尽致，色彩与花朵，创造出活泼明亮、优雅，颜色展示，2、切忌暴晒，高温，雨淋，重压，碰刮，潮湿和接触性油性物质及化学溶剂，态度，制作出一件又一件作品，异域风情、虹彩色调、快乐花朵，，结合了东方文化的沉稳意境、拉丁民族，独特的作品，其设计风格为自由空间、，4、在化妆、洗澡时也应该将手机壳放到一边，以免化妆品沐浴露等化学药剂腐蚀，(保养建议)",



In [22]:
data[5000: 5100]

['            "适用年龄": "25-29周岁",\n',
 '            "腰型": "宽松腰",\n',
 '            "廓形": "H型",\n',
 '            "领型": "其它",\n',
 '            "裙型": "一步裙",\n',
 '            "袖长": "长袖",\n',
 '            "流行元素": "纽扣",\n',
 '            "组合形式": "假两件",\n',
 '            "上市时间": "2019年冬季",\n',
 '            "材质": "聚酯纤维"\n',
 '        },\n',
 '        "ocr": "连衣裙闪光点，斜纹面料，纯色斜纹内衬，手感丝滑，亲肤穿着很享受，玫瑰金珍珠大纽扣让人忍不住多看两眼，，新优雅气质不能忽视，喇叭袖也很傲娇，直筒裙单排扣，尖领翻领设计，还有两个系带，有几分领带的即视感，清，香风再现，对称直插大口袋，很温婉的小姐姐feel，厚薄:，柔软:偏硬，版型:修身，弹性:，弹力，适中，柔软，微弹，薄款，宽大，厚款，加厚，无弹，宽松，透视，较透",\n',
 '        "reference": "粗花呢面料，质感挺括不易皱，再现经典的小香风，配上纯色斜纹内衬，手感丝滑柔软，穿着亲肤很享受。尖领翻领的设计，点缀上系带，有几分领带即视感，尽显清新雅致的气质。"\n',
 '    },\n',
 '    "289": {\n',
 '        "title": "三彩 冬季 新款 毛呢 休闲裤 直 筒裤 通勤 纯色 羊毛 裤子 长裤 女 咖啡",\n',
 '        "kb": {\n',
 '            "流行元素": "口袋",\n',
 '            "裤门襟": "拉链",\n',
 '            "面料": "羊毛",\n',
 '            "裤长": "长裤",\n',
 '            "材质": "羊毛",\n',
 '            "裤型": "直筒裤",\n',
 '            "组合形式": "单件",\n',
 '     

In [12]:
d = ''
for i in data:
    d += i

In [15]:
x = json.loads(d)

In [17]:
from collections import Counter

In [18]:
counter = Counter()

In [43]:
from collections import Counter

import numpy as np


class Vocab(object):
    PAD = 0
    SOS = 1
    EOS = 2
    UNK = 3

    def __init__(self):
        self.word2index = {}
        self.index2word = {}
        self.word2count = Counter()
        self.reserved = ['<PAD>', '<SOS>', '<EOS>', '<UNK>']
#         self.index2word = self.reserved[:]
        self.embeddings = None
        self.idx = 4
        
        
    def add_words(self, words):
        """Add a new token to the vocab and do mapping between word and index.

        Args:
            words (list): The list of tokens to be added.
        """
        ###########################################
        #          TODO: module 1 task 1          #
        ###########################################
        
        for word in words:
            self.word2index[word] = self.word2index.get(word, self.idx)
            self.index2word[self.idx] = word
            self.reserved.append(word)
            self.word2count.update(word)
            self.idx += 1
            

    def load_embeddings(self, file_path: str, dtype=np.float32) -> int:
        """Load embedding word vector.

        Args:
            file_path (str): The file path of word vector to load.
            dtype (numpy dtype, optional): Defaults to np.float32.

        Returns:
            int: Number of embedded tokens.
        """
        num_embeddings = 0
        vocab_size = len(self)
        with open(file_path, 'rb') as f:
            for line in f:
                line = line.split()
                word = line[0].decode('utf-8')
                idx = self.word2index.get(word)
                # Check whether the token is in the vocab.
                # 如果在词表中就更新embeddings
                if idx is not None:
                    vec = np.array(line[1:], dtype=dtype)
                    if self.embeddings is None:
                        # Get embedding dimension.
                        n_dims = len(vec)
                        # Initialize word vectors.
                        self.embeddings = np.random.normal(
                            np.zeros((vocab_size, n_dims))).astype(dtype)
                        pad_idx = self.word2index['<PAD>']
                        self.embeddings[pad_idx] = np.zeros(n_dims)
                    # 更新embeddings，key为对应的wordidx，value为词向量    
                    # self.embeddings --> {word_id1:word1_vec, word_idx2:word2_vec...}
                    self.embeddings[idx] = vec
                    num_embeddings += 1
        return num_embeddings

    def __getitem__(self, item):
        if type(item) is int:
            return self.index2word[item]
        return self.word2index.get(item, self.UNK)

    def __len__(self):
        return len(self.index2word)

    def size(self):
        """Returns the total size of the vocabulary"""
        return len(self.index2word)


In [1]:
for i in range(-1, 5):
    print(i)

-1
0
1
2
3
4


In [45]:
v.add_words('a b c d '.split(' '))

In [46]:
v['a']

4

In [11]:
def partition(arr,low,high): 
    i = ( low-1 )         # 最小元素索引
    pivot = arr[high]     
  
    for j in range(low , high): 
  
        # 当前元素小于或等于 pivot 
        if   arr[j] <= pivot: 
          
            i = i+1 
            arr[i],arr[j] = arr[j],arr[i] 
  
    arr[i+1],arr[high] = arr[high],arr[i+1] 
    return ( i+1 ) 
  
 
# arr[] --> 排序数组
# low  --> 起始索引
# high  --> 结束索引
  
# 快速排序函数
def quickSort(arr,low,high): 
    if low >= high: return
    
    pi = partition(arr,low,high) 
    quickSort(arr, low, pi-1) 
    quickSort(arr, pi+1, high) 
  
arr = [10, 7, 8, 9, 1, 5] 
n = len(arr) 
quickSort(arr,0,n-1) 
arr

1
2
3
4


[1, 5, 7, 8, 9, 10]